In [4]:
import sqlite3
import pandas as pd
import numpy as np
import pyproj as _proj
import os
from  os.path import join

## Collect the DRT results for simulation

In [5]:
fldr = '/media/pedro/storage/GDrive/DATA/Pedro/Professional/VLC/Projects/Vancouver/'

simul_fldr = os.path.join(fldr, 'SIMULATIONS/trunk and feeder')

log = os.path.join(simul_fldr,'experiment_list.log')

In [21]:
links = link_data()
pax_agg_kpi = [['VehicleCapacity', 'detour', 'TimeWindow','Metric','Value']]
traces_agg_kpi = [['VehicleCapacity', 'detour', 'TimeWindow','Metric','Value']]
for i in open(log, 'r').readlines():
    if i[0].isdigit():
        print (i.rstrip())
        a = collect_data(os.path.join(simul_fldr,  "bus_passengers_" + i.rstrip() + '.sqlite'))
        vc, det, tw = i.rstrip().split("-")
        for metric, value in pax_statistics(a):
            pax_agg_kpi.append([str(vc), str(det), str(tw), str(metric), str(value)])
        
        for metric, value in trace_statistics(a, links):
            traces_agg_kpi.append([str(vc), str(det), str(tw), str(metric), str(value)])
        

3-0-10
3-0-15
3-0-20
3-0-25
3-0-30
3-2-10
3-2-15
3-2-20
3-2-25
3-2-30
3-4-10
3-4-15
3-4-20
3-4-25
3-4-30
3-6-10
3-6-15
3-6-20
3-6-25
3-6-30
3-8-10
3-8-15
3-8-20
3-8-25
3-8-30
3-10-10
3-10-15
3-10-20
3-10-25
3-10-30
4-0-10
4-0-15
4-0-20
4-0-25
4-0-30
4-2-10
4-2-15
4-2-20
4-2-25
4-2-30
4-4-10
4-4-15
4-4-20
4-4-25
4-4-30
4-6-10
4-6-15
4-6-20
4-6-25
4-6-30


In [22]:
with open(os.path.join(fldr, 'SIMULATIONS/trunk and feeder/passenger_metrics.csv'), 'w') as p:
    for l in pax_agg_kpi:
        p.write(','.join(l) + '\n')

with open(os.path.join(fldr, 'SIMULATIONS/trunk and feeder/trace_metrics.csv'), 'w') as p:
    for l in traces_agg_kpi:
        p.write(','.join(l) + '\n')

In [14]:
class dt:
    def __init__(self):
        traces = None
        pax = None
        events = None
        
def collect_data(file, items=['pax', 'traces', 'events']):
    a = dt()
    results = sqlite3.connect(file)
    
    st = 7.5 * 3600
    nd = 8.5 * 3600
    pax_qry = 'select * from passengers where required_pickup >={} AND required_pickup <{}'.format(st,nd)
    traces_qry = 'select * from vehicle_traces where instant >={} AND instant <{}'.format(st,nd)
    events_qry = 'select * from events'
    if 'traces' in items:
        traces = pd.read_sql(traces_qry, results)
        traces.sort_values(by=['vehicle_id', 'instant'], inplace=True)
        a.traces = traces
    if 'pax' in items:
        pax = pd.read_sql(pax_qry, results)
        a.pax = pax
    return a
    

In [12]:
def pax_statistics(data):
    pax_data = data.pax.assign(detour=data.pax.actual_travel_time - a.pax.expected_travel_time)
    
    p = pax_data.shape[0]
    metrics = [('total passengers', p)]
    metrics.append(('total pax.h detour', round(pax_data.detour.sum()/3600, 2)))
    metrics.append(('Average pax.h detour', round(pax_data.detour.sum()/(3600 * p), 4)))
    s = pax_data.extra_stops.sum()
    metrics.append(('Total extra passenger stops', s))
    metrics.append(('Average extra stops per passenger', round(s/p, 3)))
    return metrics

In [6]:
def link_data():
    lineslyr = sqlite3.connect(join(fldr, "Network/final_layer.gpkg"))
    links = pd.read_sql_query("SELECT LENGTH, A_Node, B_Node FROM links", lineslyr)
    links = links.assign(link = links.A_Node * 10000 + links.B_Node)
    links.loc[:, 'link'] = links.link.values.astype(np.int32)[:]
    return links

In [20]:
def trace_statistics(data, links):
    metrics = []
    traces = data.traces
    
    traces = traces.assign(to_node = traces.node_id.shift(-1))
    traces = traces.assign(to_veh_id = traces.vehicle_id.shift(-1))
    traces.loc[traces.to_veh_id != traces.vehicle_id,'to_node'] = np.nan
    traces = traces.assign(link = traces.node_id * 10000 + traces.to_node)
    traces.loc[:, 'link'] = traces.link.values.astype(np.int32)[:]
    traces.dropna(inplace=True)

    with_distances = traces.merge(links, on='link')
    with_distances.LENGTH.isna().any()
    
    empty = with_distances.loc[with_distances.occupancy==0, 'LENGTH'].sum()
    metrics.append(['Dead heading', empty])
    metrics.append(['Total Veh.km', with_distances.LENGTH.sum()])
    
    mx_oc = traces.occupancy.max()
    metrics.append(['Maximum occupancy',mx_oc])
    
    length_full = with_distances.loc[with_distances.occupancy==mx_oc, 'LENGTH'].sum()
    metrics.append(['Veh.km with maximum occupancy',length_full])
    
    return metrics
